In [1]:
import random
from math import log2
from scipy.sparse import csc_matrix, eye, find
import numpy as np
import time
import sys
import pandas as pd

In [2]:
class RANDU:
    def __init__(self, mul, mod,seed):
        self.mul = mul
        self.mod = mod
        self.currvalue = seed
        self.seed = seed
    def rand(self):
        self.currvalue = (self.currvalue*self.mul) % self.mod
        return self.currvalue
    def rewind(self):
        self.currvalue = self.seed

def matrixpower(M, p):
    k = M.shape[0]
    if p ==0:
        return eye(k)
    M1 = M.copy()
    for _ in range(p-1):
        M1 = M1.dot(M)
        if _ % 50 == 0:
            M1 = matrixmod2(M1)
    return M1.copy()

def quickpow (M, pow):
  result = eye(M.shape[0]).tocsc()
  while pow > 0:
    if pow % 2 == 1:
      result = result.dot(M)
    result = matrixmod2(result)
    M = M.dot(M.copy())
    M = matrixmod2(M)
    pow >>= 1
  return result.copy()

def matrixmod2(M):
    rows,cols = M.nonzero()
    for row,col in zip(rows,cols):
        M[row,col] %= 2
    return M.copy()
class Generator:
    def __init__(self, j, k, seed):
        self.j = j
        self.k = k
        self.g = RANDU(65539, 2**31, seed)
        self.currvalue = [self.g.rand() for _ in range(k)]
        self.initial = self.currvalue.copy()
        self.seed = seed
        self.currpos = 0
    def rand(self):
        self.currvalue[self.currpos] = self.currvalue[self.currpos] ^ self.currvalue[(self.currpos-self.j) % self.k]
        self.currpos = (self.currpos+1)%self.k
        val = self.currvalue[self.currpos]
        return val
    def rewind(self):
        self.currvalue = self.initial.copy()
        self.currpos = 0
    
    def jumpahead(self, jlen):
        A = csc_matrix(([1]*(self.k+1), ([i for i in range(self.k)]+[self.k-1], [i for i in range(1,self.k)]+[0, self.k-self.j])), shape=(self.k,self.k), dtype="int")
        m, r = divmod(jlen, self.k)
        dif =  self.k - self.j - r
        if dif>0:
            B = matrixpower(A, r)
            print("B is powered in {} degree".format(r))
            C = B.copy()
            C = C.dot(matrixpower(A, dif))
            print("C is powered in {} degree".format(self.k - self.j))
        else:
            C = matrixpower(A, self.k - self.j)
            print("C is powered in {} degree".format(self.k - self.j))
            B = C.copy()
            B = B.dot(matrixpower(A, -dif))
            print("B is powered in {} degree".format(r))
        C = C + eye(self.k)
        C = matrixmod2(C)
        C = quickpow(C, m)
        C = matrixmod2(C)
        print("C+E is powered in {} degree".format(m))
        D = B.dot(C)
        D = matrixmod2(D)
        invres = 0
        res = 0
        bits = csc_matrix((self.k,1), dtype = "int")
        for _ in range(sys.getsizeof(int)):
            for k in range(self.k):
                bits[k] = self.currvalue[k]%2
                self.currvalue[k]>>=1
            b = int(D.dot(bits)[0,0]) % 2
            invres = b | (invres << 1)
        self.currvalue = self.initial.copy()
        for _ in range(sys.getsizeof(int)):
            res = (res << 1) | (invres % 2)
            invres>>=1
        return res
    def just_power(self, n):
        A = csc_matrix(([1]*(self.k+1), ([i for i in range(self.k)]+[self.k-1], [i for i in range(1,self.k)]+[0, self.k-self.j])), shape=(self.k,self.k), dtype="int")
        quickpow(A,n)

In [3]:
seed = 24121
jlen = 1000000000
gen = Generator(24, 55, seed)

In [4]:
gen.rewind()

In [5]:
start = time.time()
a = gen.jumpahead(jlen)
print(a)
print("it takes {} seconds".format(time.time() - start))

B is powered in 10 degree
C is powered in 31 degree
C+E is powered in 18181818 degree
338321978
it takes 6.69215989112854 seconds


c:\users\acer\pycharmprojects\computationalworkshop\venv\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [6]:
start = time.time()
a = gen.jumpahead(1000)
for _ in range(999):
    gen.rand()
print("it takes {} seconds".format(time.time() - start))
b = gen.rand()
print("Results are {} and {}".format(a,b))

B is powered in 10 degree
C is powered in 31 degree
C+E is powered in 18 degree
it takes 1.7311928272247314 seconds
Results are 996197336 and 996197336


In [7]:
start = time.time()
gen.just_power(jlen)
print("it takes {} seconds".format(time.time() - start))

it takes 5.169907808303833 seconds


### Основная идея быстрого возведения в степень:
### Матрица $A$ является фробениусовой матрицей, внизу единица стоит в столбце с номером k-j, если считать от нуля.
### Её характеристический многочлен $p(x) = x^k-x^{k-j}-1$.
### По теореме Гамильтона-Кэли $p(A) = A^k-A^{k-j}-E=0$
### Тогда $A^k = A^{k-j}+E$
### Тогда возведение в степень $n, \ r = n \% k, \ m = n // k$
### $A^{n} = A^{k*m+r} = A^r(A^k)^m = A^r(A^{k-j}+E)^m$
### Возводить в m-ую степень уже с помощью двоичного представления числа m